## Fig 1 - EMG LEARNING

Figures generated in this notebook use:

* EMG Direction Tuning
* Task Error (Mean Error)

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import wilcoxon as wilcoxon

# meta analysis functions
import sys
sys.path.append('/code/')
from util import analysis
from util import plotting
from util import util_continuous as utils


In [ ]:
PATH = 'data/'

Get Data Files

In [ ]:
# load the binned and target-tuned velocities and angles
with open(PATH + 'emg-direction-tuning-data/velocities-emgs-target-seperated-ntarg-10.pickle', 'rb') as handle:
    vels_binned, angles_binned, times_binned, emg_times_binned, emg_targ_binned, keys, conds, vint_epoch, vint_len, emg_sample_len, n_targs = pickle.load(handle)

# load the emg data that has been separated into early and late portions
    
with open(PATH + 'emg-direction-tuning-data/emg-target-tuning-early-late-ntarg-10.pickle','rb') as handle:
    emgs_dir_early, emgs_dir_late, emgs_dir_early_std, emgs_dir_late_std, tun_params_early, tun_params_late, md_early, md_late, pd_early, pd_late, num_targ_trials, emg_sample_len, n_ch, n_targs, targ_array = pickle.load(handle)

# load the order of the trials in terms of when the subject saw the trial
with open(PATH + 'trial-related-data/decoder_trials_in_order.pickle','rb') as handle:
    trials_list, conds_trial_list = pickle.load(handle)

# using the task error that calculates time domain in cm
with open(PATH + 'time-domain-error/time-domain-error-30sec-in-cm.pkl','rb') as handle:
    td_error, td_error_first, td_error_last, t0_start, t0_end, t1_end, td_diff, td_diff_slow, td_diff_fast, td_diff_pos, td_diff_neg, td_diff_pD3, td_diff_pD4 = pickle.load(handle)


Table of variables used 

| Variable | Shape | Meaning |
| --- | --- | --- |
| emgs_dir_early | (14, 2, 8, 64, 8) | mean emg amplitude by target direction in early part of trials |
| emgs_dir_late | (14, 2, 8, 64, 8) | mean emg amplitude by target direction in late part of trials |
| emgs_dir_diff | (14, 2, 8, 64) | norm difference between emgs_dir_late - emgs_dir_early  |
| emgs_dir_diff_conds | (14, 2, 8) | emgs_dir_diff average across conditions |
| emgs_dir_diff_trial | (14, 2, 8) | emgs_dir_diff_conds in order of trials seen to subject (conds_trial_list) |
| conds_trial_list | (2, 14, 8) | order of trials shown to subjects |

## WITHIN TRIAL LEARNING

### Fig EMG-b: example subject/EMG  channel change during a trial

Using Subject 108 (subject has well-tuned EMG channels)

In [ ]:
ex_subj = 3
print(keys[ex_subj])

Example subject, all trials

In [ ]:
pd_early[0, :, :, 0]

In [ ]:
# Import seaborn
import seaborn as sns

sns.set_theme(style="ticks", rc=utils.sns_custom_params, font_scale=0.6)


In [ ]:
fig, ax = plt.subplots(14, 5, sharey=False, figsize = (20, 28))
for iSubj in range(14):
    ich = 0
    for ch in [10, 13, 36, 39, 62]:
        axs = ax[iSubj, ich]
        ich = ich + 1 # for next time
        axs.set_title("ch " + str(ch + 1), fontsize = 20)
        
        # take the early emg signals for only one subject, separate by channel
        # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
        early_25, early_50, early_75 = np.percentile(emgs_dir_early[iSubj, :, :, ch, :], [25, 50, 75], axis = (0, 1))
        axs.plot(np.linspace(0, 360, n_targs), early_50, '-o', color = 'blue')
        lower_end = early_25 
        upper_end = early_75 
        axs.fill_between(np.linspace(0, 360, n_targs), lower_end, upper_end, color = 'blue', alpha = 0.2)

        # take the late emg signals for only one subject, separate by channel
        # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
        late_25, late_50, late_75 = np.percentile(emgs_dir_late[iSubj, :, :, ch, :], [25, 50, 75], axis = (0, 1))
        axs.plot(np.linspace(0, 360, n_targs), late_50, '-o', color = utils.colors['late'])
        lower_end = late_25 
        upper_end = late_75 
        axs.fill_between(np.linspace(0, 360, n_targs), lower_end, upper_end, color = utils.colors['late'], alpha = 0.2)

        # also plot the PD early mean 
        pd_early_50 = np.mean(pd_early[iSubj, :, :, ch], axis = (0, 1)) * 180/np.pi
        pd_late_50 = np.mean(pd_late[iSubj, :, :, ch], axis = (0, 1)) * 180/np.pi

        axs.vlines(pd_early_50, 0, max(upper_end), colors = 'blue')
        axs.vlines(pd_late_50, 0, max(upper_end), colors = utils.colors['late'])


In [ ]:
# image_format = 'pdf' # e.g .png, .svg, etc.
# image_name = 'sfig2-emg-chs-aggregate.pdf'
# #s0.pdf
# PATH = '/Users/mmadduri/OneDrive - UW/PhD_Research/Figures/myo-coadapt-2023/nov2023/fig1/python-figs/'
# fig.savefig(PATH + image_name, format=image_format, dpi=300)

In [ ]:
fig, ax = plt.subplots(13, 5, sharey=False, figsize = (20, 28))
for iCh in range(n_ch-1):
   ax = ax.flatten()
   axs = ax[iCh]
   axs.set_title("ch " + str(iCh + 1), fontsize = 20)
      
   # take the early emg signals for only one subject, separate by channel
   # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
   early_25, early_50, early_75 = np.percentile(emgs_dir_early[ex_subj, :, :, iCh, :], [25, 50, 75], axis = (0, 1))
   axs.plot(np.linspace(0, 360, n_targs), early_50, '-o', color = 'blue')
   lower_end = early_25 
   upper_end = early_75 
   axs.fill_between(np.linspace(0, 360, n_targs), lower_end, upper_end, color = 'blue', alpha = 0.2)


   # take the late emg signals for only one subject, separate by channel
   # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
   late_25, late_50, late_75 = np.percentile(emgs_dir_late[ex_subj, :, :, iCh, :], [25, 50, 75], axis = (0, 1))
   axs.plot(np.linspace(0, 360, n_targs), late_50, '-o', color = utils.colors['late'])
   lower_end = late_25 
   upper_end = late_75 
   axs.fill_between(np.linspace(0, 360, n_targs), lower_end, upper_end, color = utils.colors['late'], alpha = 0.2)

   # also plot the PD early mean 
   pd_early_50 = np.mean(pd_early[ex_subj, :, :, iCh], axis = (0, 1)) * 180/np.pi
   pd_late_50 = np.mean(pd_late[ex_subj, :, :, iCh], axis = (0, 1)) * 180/np.pi

   axs.vlines(pd_early_50, 0, max(upper_end), colors = 'blue')
   axs.vlines(pd_late_50, 0, max(upper_end), colors = utils.colors['late'])

   

fig.tight_layout()
ax[5].set_ylabel(keys[ex_subj], fontsize = 20, color = 'red')
ax[15].set_ylabel("all blocks, all trials", fontsize = 20, color = 'red')

In [ ]:
# image_format = 'pdf' # e.g .png, .svg, etc.
# image_name = 'sfig2-emg-chs-s' + str(ex_subj) + '.pdf' 
# #s0.pdf
# PATH = '/Users/mmadduri/OneDrive - UW/PhD_Research/Figures/myo-coadapt-2023/nov2023/fig1/python-figs/'
# fig.savefig(PATH + image_name, format=image_format, dpi=300)

Example subject, PD, all trials

In [ ]:
# fig, ax = plt.subplots(13, 5, sharey=False, figsize = (20, 28))
# for iCh in range(n_ch-1):
#    ax = ax.flatten()
#    axs = ax[iCh]
#    axs.set_title("ch " + str(iCh + 1), fontsize = 20)

   
#    # take the early emg signals for only one subject, separate by channel
#    # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
#    early_25, early_50, early_75 = np.percentile(emgs_dir_early[ex_subj, :, :, iCh, :], [25, 50, 75], axis = (0, 1))
#    axs.plot(np.linspace(0, 360, n_targ), early_50, '-o', color = colors['early'])
#    upper_end = early_25 
#    lower_end = early_75 
#    axs.fill_between(np.linspace(0, 360, n_targ), lower_end, upper_end, color = colors['early'], alpha = 0.2)

#    # take the late emg signals for only one subject, separate by channel
#    # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
#    late_25, late_50, late_75 = np.percentile(emgs_dir_late[ex_subj, :, :, iCh, :], [25, 50, 75], axis = (0, 1))
#    axs.plot(np.linspace(0, 360, n_targ), late_50, '-o', color = colors['late'])
#    upper_end = late_25 
#    lower_end = late_75 
#    axs.fill_between(np.linspace(0, 360, n_targ), lower_end, upper_end, color = colors['late'], alpha = 0.2)
   

# fig.tight_layout()
# ax[5].set_ylabel(keys[ex_subj], fontsize = 20, color = 'red')
# ax[15].set_ylabel("all blocks, all trials", fontsize = 20, color = 'red')


Show the example subject and one trial of that example subject

In [ ]:
early_50.shape

In [ ]:
# # pick the first trial as an example trial
# ex_subj = 3
# ex_block = 1
# ex_cond = 2
# ex_trial = conds_trial_list[ex_block, ex_subj, ex_cond] 

# for iCh in range(n_ch - 1):
#     fig, ax = plt.subplots(1, 1, figsize = (1, 1))
#     ax.set_title("Ch: " + str(iCh))
#     early_50 = emgs_dir_early[ex_subj, ex_block, ex_trial, iCh, :]
#     ax.plot(np.linspace(0, 360, n_targs), early_50, '-o', color = 'blue')
#     late_50 = emgs_dir_late[ex_subj, ex_block, ex_trial, iCh, :]
#     ax.plot(np.linspace(0, 360, n_targs), late_50, '-o', color = utils.colors['late'])



In [ ]:
# pick the first trial as an example trial
ex_subj = 2
ex_block = 1
ex_cond = 3
ex_trial = conds_trial_list[ex_block, ex_subj, ex_cond] 

fig, ax = plt.subplots(13, 5, sharey=False, figsize = (20, 28))
for iCh in range(n_ch-1):
   ax = ax.flatten()
   axs = ax[iCh]
   axs.set_title("ch " + str(iCh + 1), fontsize = 20)

   
   # take the early emg signals for only one subject, separate by channel
   # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
   early_50 = emgs_dir_early[ex_subj, ex_block, ex_trial, iCh, :]
   error_50 = emgs_dir_early_std[ex_subj, ex_block, ex_trial, iCh, :]
   axs.plot(np.linspace(0, 360, n_targs), early_50, '-o', color = 'blue')

   # including upper and lower limits
   # axs.errorbar(np.linspace(0, 360, n_targs), early_50, xerr=0, yerr=error_50, color = 'blue')

   # take the late emg signals for only one subject, separate by channel
   # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
   late_50 = emgs_dir_late[ex_subj, ex_block, ex_trial, iCh, :]
   error_late_50 = emgs_dir_late_std[ex_subj, ex_block, ex_trial, iCh, :]
   axs.plot(np.linspace(0, 360, n_targs), late_50, '-o', color = utils.colors['late'])
   # including upper and lower limits
   # axs.errorbar(np.linspace(0, 360, n_targs), late_50, xerr=0, yerr=error_late_50, color = utils.colors['late'])



fig.tight_layout()
ax[5].set_ylabel(keys[ex_subj], fontsize = 20, color = 'red')
ax[15].set_ylabel("BLOCK: " + str(ex_block + 1) + " TRIAL: " + str(ex_cond + 1), fontsize = 20, color = 'red')



Example Subject, block 1 only

In [ ]:
# ex_block = 0
# fig, ax = plt.subplots(13, 5, sharey=False, figsize = (20, 28))
# for iCh in range(n_ch-1):
#    ax = ax.flatten()
#    axs = ax[iCh]
#    axs.set_title("ch " + str(iCh + 1), fontsize = 20)

   
#    # take the early emg signals for only one subject, separate by channel
#    # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
#    early_25, early_50, early_75 = np.percentile(emgs_dir_early[ex_subj, ex_block, :, iCh, :], [25, 50, 75], axis = (0)) 
#    # axis = 0, percentile across the 8 conditions in block 1 only
#    axs.plot(np.linspace(0, 360, n_targs), early_50, '-o', color = 'blue')
#    upper_end = early_25 
#    lower_end = early_75 
#    axs.fill_between(np.linspace(0, 360, n_targs), lower_end, upper_end, color = 'blue', alpha = 0.2)

#    # take the late emg signals for only one subject, separate by channel
#    # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
#    late_25, late_50, late_75 = np.percentile(emgs_dir_late[ex_subj, ex_block, :, iCh, :], [25, 50, 75], axis = (0))
#    # axis = 0, percentile across the 8 conditions in block 1 only
#    axs.plot(np.linspace(0, 360, n_targs), late_50, '-o', color = utils.colors['late'])
#    upper_end = late_25 
#    lower_end = late_75 
#    axs.fill_between(np.linspace(0, 360, n_targs), lower_end, upper_end, color = utils.colors['late'], alpha = 0.2)
   

# fig.tight_layout()
# ax[5].set_ylabel(keys[ex_subj], fontsize = 20, color = 'red')
# ax[15].set_ylabel("BLOCK: " + str(ex_block + 1), fontsize = 20, color = 'red')


In [ ]:
label_size = 8 
plt.rcParams['axes.titlesize'] = 6
plt.rcParams['axes.labelsize'] = label_size

Add the specific figures to the subplots

* subj, block, trial, ch = 3, 0, 3, 50
* subj, block, trial, ch = 2, 0, 2, 13


In [ ]:
# ex_subj = 3
# ex_block = 1
# ex_cond = 2
# ex_trial = conds_trial_list[ex_block, ex_subj, ex_cond] 
# ex_ch = 22

ex_subj = 2
ex_block = 1
ex_cond = 3
ex_trial = conds_trial_list[ex_block, ex_subj, ex_cond] 
ex_ch = 14

In [ ]:
## SETUP THE FIGURE HERE
## HAVE TO RE-REUN FROM HERE TO "CLEAR" THE PLOT
fig_emg = plt.figure(figsize = (6, 4), layout='constrained') # set the total figure size

mosaic = """
    bca
    efd
    """

# set up the axes
ax_dict = fig_emg.subplot_mosaic(mosaic)
for ii in ax_dict:
    plotting.remove_and_set_axes(ax_dict[ii], bottom=True, left=True)
    ax_dict[ii].tick_params(axis='both', which='major', labelsize = label_size - 2)
    ax_dict[ii].tick_params(axis='both', which='minor', labelsize = label_size - 2)
fig_emg.patch.set_facecolor('white')

# across trial learning
# a
ax_dict['a'].set_xlabel("Trials")
ax_dict['a'].set_ylabel(r'|$\Delta$ EMG Amplitude| ($\mu V$)')
ax_dict['a'].set_title("|$\Delta$ EMG| Across Trials ")


# d
ax_dict['d'].set_xlabel("Time (minutes)")
ax_dict['d'].set_ylabel("Mean Error (cm)")
ax_dict['d'].set_title("Performance Across Blocks")

# within-trial learning
# b 
ax_dict['b'].set_xlabel("Direction (degrees)")
ax_dict['b'].set_ylabel(r'EMG Amplitude ($\mu V$)')
ax_dict['b'].set_title("Sample Subject | Trial " + str(ex_cond + 1))

# c
ax_dict['c'].set_xlabel("Direction (degrees)")
ax_dict['c'].set_ylabel(r'EMG Amplitude ($\mu V$)')
ax_dict['c'].set_title("Sample Subject | All Trials ")

# e
ax_dict['e'].set_xlabel("Subjects")
ax_dict['e'].set_ylabel("|$\Delta$ EMG Amplitude|$_2$")
ax_dict['e'].set_title('$\Delta$ EMG Across Subjects')


# e
ax_dict['f'].set_xlabel("Subjects")
ax_dict['f'].set_ylabel("$|PD_{ch, late} - PD_{ch, early}|$")
ax_dict['f'].set_title('$\Delta$ EMG Across Subjects')


fig_emg.patch.set_facecolor('white')


In [ ]:
## FIGURE B - example subject, example channel, only one trial
fig_ax = 'b'
axs = ax_dict[fig_ax]

# take the early emg signals for only one subject, separate by channel
# find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
early_50 = emgs_dir_early[ex_subj, ex_block, ex_trial, ex_ch, :]
axs.plot(np.linspace(0, 360, n_targs), early_50, '-o', color = 'blue', ms = 3, label = 'early')

error_50 = emgs_dir_early_std[ex_subj, ex_block, ex_trial, ex_ch, :]
# axs.errorbar(np.linspace(0, 360, n_targs), early_50, xerr=0, yerr=error_50, color = 'blue', lw = 0.5, zorder=10)


# take the late emg signals for only one subject, separate by channel
 # find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
late_50 = emgs_dir_late[ex_subj, ex_block, ex_trial, ex_ch, :]
axs.plot(np.linspace(0, 360, n_targs), late_50, '-o', color = utils.colors['late'], ms = 3, label= 'late')

late_error_50 = emgs_dir_late_std[ex_subj, ex_block, ex_trial, ex_ch, :]
# axs.errorbar(np.linspace(0, 360, n_targs), late_50, xerr=0, yerr=error_late_50, 
            #  color = utils.colors['late'], label = 'late', lw = 0.5)


axs.legend()

fig_emg

In [ ]:
## FIGURE C
fig_ax = 'c'

early_25, early_50, early_75 = np.percentile(emgs_dir_early[ex_subj, :, :, ex_ch, :], [25, 50, 75], axis = (0, 1)) 
# axis = 0, 1, across all conditions
ax_dict[fig_ax].plot(np.linspace(0, 360, n_targs), early_50, '--', color = 'blue')
lower_end = early_25 
upper_end = early_75 
ax_dict[fig_ax].fill_between(np.linspace(0, 360, n_targs), lower_end, upper_end, color = 'blue', alpha = 0.2)

# take the late emg signals for only one subject, separate by channel
# find the 25, 50, 70 percentiles across ALL trials (both blocks, so 16 trials total)
late_25, late_50, late_75 = np.percentile(emgs_dir_late[ex_subj, :, :, ex_ch, :], [25, 50, 75], axis = (0, 1))
ax_dict[fig_ax].plot(np.linspace(0, 360, n_targs), late_50, color = utils.colors['late'])
lower_end = late_25 
upper_end = late_75 
ax_dict[fig_ax].fill_between(np.linspace(0, 360, n_targs), lower_end, upper_end, color = utils.colors['late'], alpha = 0.2)


# also plot the PD early and late vertical lines
pd_early_50 = np.mean(pd_early[ex_subj, :, :, ex_ch], axis = (0, 1)) * 180/np.pi
pd_late_50 = np.mean(pd_late[ex_subj, :, :, ex_ch], axis = (0, 1)) * 180/np.pi

ax_dict[fig_ax].vlines(pd_early_50, -0, max(upper_end), colors = 'blue', lw = 1, ls = '--')
ax_dict[fig_ax].vlines(pd_late_50, -0, max(upper_end), colors = utils.colors['late'],lw = 1)

# ax_dict[fig_ax].set_ylim(10, 60)

ax_dict['b'].get_shared_y_axes().join(ax_dict['b'], ax_dict['c'])
# ax_dict['c'] = plt.subplot(sharey=ax_dict['b'])

fig_emg


Figure e - emg amplitudes across subject

In [ ]:
emgs_dir_early.shape

In [ ]:
fig_ax = 'e'
axs = ax_dict[fig_ax]

# emgs_dir_late and emg_dir_early = (14, 2, 8, 64, n_targs = 10) = n_keys, n_blocks, n_conds, n_emg, n_targs
# taking the norm difference between late and early emg amplitude 
emgs_dir_diff = np.linalg.norm(emgs_dir_late - emgs_dir_early, ord = 2, axis = -1)
assert(emgs_dir_diff.shape == (utils.n_keys, utils.n_blocks, utils.n_conds, n_ch))

# then taking the mean for each subject and for each channel
# to see if, per subject, there is a significant change across channels per subject
emgs_dir_diff_mean = np.mean(emgs_dir_diff[:, :, :, :-1], axis = (1, 2)) # :-1 to remove the last channel (64) that is just noise because of the differential signal
assert(emgs_dir_diff_mean.shape == (utils.n_keys, n_ch-1))


ylim = 120
y_sign = ylim*.8

for iK, key in enumerate(keys):
    data = emgs_dir_diff_mean[iK]
    data_groups = [data] 
    data_pos = [iK+1]
    bplot = axs.boxplot(data_groups, 
                            showfliers=False,
                            patch_artist=True,
                            positions=data_pos,
                            boxprops=dict(edgecolor="none", alpha = 0.5),
                            medianprops=dict(color='k', lw=1), 
                            widths = 0.8)
        
    if utils.colors is not None:
       for patch, color in zip(bplot['boxes'], 
                                [utils.colors['emg']]):
                patch.set_facecolor(color)

    # see significance assuming late > early
    # wilcoxon is comparing across channels for each subject, so N = 64 per subject
    assert(emgs_dir_diff_mean[iK].flatten().shape == (n_ch - 1, ))
    # compare distribution to zeros - tested against a gaussian distribution with mean = 0
    w = wilcoxon(data.flatten(), np.zeros(len(emgs_dir_diff_mean[iK])),
                 alternative='greater')
    pv1 = w.pvalue
    print(key)
    print(w)
    # bonferroni correction, p' = p*sqrt(# test) where # tests here = 14
    pv1_bon = np.sqrt(14)*pv1
    print("p - bonferroni correction: ", pv1_bon)
    

    if pv1_bon <= 0.05:
        asterix =   "*"
        # Add significance asterisk and line between boxplots
        axs.text((max(data_pos) + min(data_pos))/2,  y_sign , asterix, ha='center', va='center', fontsize=8, color = 'k')
    


axs.set_xticks(np.linspace(1, 14, utils.n_keys), utils.subj_labels, rotation=40, fontsize = 6);
axs.set_ylim(-5, ylim)
axs.hlines(0, 0, utils.n_keys, color = 'gray', alpha = 1, ls = '--', lw = 0.5)

fig_emg


In [ ]:
# supplementary plot if needed -- wanted to compare the change in the 
fig, axs = plt.subplots(1, 1, figsize = (1, 1))
# axs = ax_dict[fig_ax]

# compare early and late
emgs_dir_early_med = np.mean(emgs_dir_early, axis = (1, 2, -1))
emgs_dir_late_med = np.mean(emgs_dir_late, axis = (1, 2, -1))

ylim = 80
y_sign = ylim*.8

for iK, key in enumerate(keys):
    # data = emgs_dir_diff_med[iK]
    data_groups = [emgs_dir_early_med[iK],emgs_dir_late_med[iK]] 
    data_pos = [iK, iK + 0.4]
    bplot = axs.boxplot(data_groups, 
                            showfliers=False,
                            patch_artist=True,
                            positions=data_pos,
                            boxprops=dict(edgecolor="none", alpha = 0.5),
                            medianprops=dict(color='k', lw=1), 
                            widths = 0.4)
        
    if utils.colors is not None:
       for patch, color in zip(bplot['boxes'], 
                                [utils.colors['emg']]):
                patch.set_facecolor(color)

    # see significance assuming late > early
    # wilcoxon is comparing across channels for each subject, so N = 64 per subject
    assert(emgs_dir_diff_mean[iK].flatten().shape == (n_ch - 1, ))
    # compare distribution to zeros - tested against a gaussian distribution with mean = 0
    w = wilcoxon(emgs_dir_early_med[iK].flatten(), emgs_dir_late_med[iK].flatten(),
                 alternative='two-sided')
    pv1 = w.pvalue
    print(key)
    print(w)
    # bonferroni correction, p' = p*sqrt(# test) where # tests here = 14
    pv1_bon = np.sqrt(14)*pv1
    print("p - bonferroni correction: ", pv1_bon)

    if pv1_bon <= 0.05:
        asterix =   "*"
        # Add significance asterisk and line between boxplots
        axs.text((max(data_pos) + min(data_pos))/2,  y_sign , asterix, ha='center', va='center', fontsize=8, color = 'k')
    


axs.set_xticks(np.linspace(0, 13, utils.n_keys), utils.subj_labels, rotation=40, fontsize = 6);
axs.set_ylim(-5, ylim)
axs.hlines(0, 0, utils.n_keys, color = 'gray', alpha = 1, ls = '--', lw = 0.5)


In [ ]:
def subtract_angles(angle1, angle2):
    """
    Subtract angle2 from angle1, accounting for wraparound.

    Parameters:
        angle1, angle2: NumPy arrays of angles in degrees

    Returns:
        Resulting angles in degrees (NumPy array) within the range [0, 360).
    """
    result = angle1 - angle2
    result %= 360  # Use modulo to ensure the result is within [0, 360)
    result[result > 180] -= 360  # Adjust if result is greater than 180 degrees
    result[result < -180] += 360  # Adjust if result is less than -180 degrees
    return np.abs(result)


In [ ]:
# figure f - change in PD

# preferred direction is one number per trial, 
pd_diff = subtract_angles(pd_late * 180/np.pi, pd_early * 180/np.pi)
assert(pd_diff.all() <= 180)
assert(pd_diff.shape == (utils.n_keys, utils.n_blocks, utils.n_conds, n_ch))
# take the mean PD across trials 
pd_diff_mean = np.mean(pd_diff[:, :, :, :-1], axis = (1, 2))
assert(pd_diff_mean.shape == (utils.n_keys, n_ch-1))

fig_ax = 'f'
axs = ax_dict[fig_ax]

ylim = 180

for iK, key in enumerate(keys):
    data = pd_diff_mean[iK]
    data_groups = [data]
    data_pos = [iK+1]
    bplot = axs.boxplot(data_groups, 
                            showfliers=False,
                            patch_artist=True,
                            positions=data_pos,
                            boxprops=dict(edgecolor="none", alpha=0.5),
                            medianprops=dict(color='k', lw=1), widths = 0.6)
    
    if utils.colors is not None:
       for patch, color in zip(bplot['boxes'], 
                                [utils.colors['emg']]):
                patch.set_facecolor(color)

    # see significance assuming late > early
    w = wilcoxon(data.flatten(), np.zeros(len(data)), alternative='greater')
    pv1 = w.pvalue
    print(key)
    print(w)
    # bonferroni correction, p' = p*sqrt(# test) where # tests here = 14
    pv1_bon = np.sqrt(14)*pv1
    print("p - bonferroni correction: ", pv1_bon)

    if pv1_bon <= 0.05:
        asterix =   "*"
        # Add significance asterisk and line between boxplots
        axs.text((max(data_pos) + min(data_pos))/2, ylim - 50, asterix, 
                 ha='center', va='center', fontsize=8, color = 'black')
    

axs.set_xticks(np.linspace(1, utils.n_keys, utils.n_keys), utils.subj_labels, rotation=40, fontsize = 6);
axs.hlines(0, 0, utils.n_keys, color = 'gray', alpha = 1, ls = '--', lw = 0.5)
axs.set_ylim(-5, ylim)

fig_emg

In [ ]:
emgs_dir_diff.shape

In [ ]:
fig_ax = 'a'

# emgs_dir_diff is the difference in emg 
# take average across all channels
emgs_dir_diff_conds = np.mean(emgs_dir_diff, axis = -1)
assert(emgs_dir_diff_conds.shape == (utils.n_keys, utils.n_blocks, utils.n_conds))

# arrange emgs_dir_diff_trials in order of the trials
emgs_dir_diff_trials = np.zeros((utils.n_keys, utils.n_blocks, utils.n_conds))
for iK in range(utils.n_keys):
    for iB in range(utils.n_blocks):
        for iC in range(utils.n_conds):
            emgs_dir_diff_trials[iK, iB, iC] = emgs_dir_diff_conds[iK, iB, conds_trial_list[iB, iK, iC]]

# flatten across blocks
emgs_dir_diff_trials_append = np.append(emgs_dir_diff_trials[:, 0], emgs_dir_diff_trials[:, 1], axis = 1)
# confirm that the data is flattened across blocks and conditions correctly
assert(emgs_dir_diff_trials_append.shape == (utils.n_keys, utils.n_blocks * utils.n_conds))

emg_diff_trial_first = emgs_dir_diff_trials_append[:, 0]
emg_diff_trial_last = emgs_dir_diff_trials_append[:, -1]
assert(emg_diff_trial_first.shape == emg_diff_trial_last.shape == (utils.n_keys, ))
w = wilcoxon(emg_diff_trial_first, emg_diff_trial_last)
print(w)

plotting.plot_significance(w.pvalue, data1 = emg_diff_trial_first, data2 = emg_diff_trial_last, 
                           data_pos=[1, 16], fig=fig_emg, ax=ax_dict[fig_ax], y_bar=-120, y_asterix=-120, fontsize=8)

# take the interquartile distribution across subjects (N = 14) 
emg_diff_25, emg_diff_50, emg_diff_75 = np.percentile(emgs_dir_diff_trials_append, [25, 50, 75],  axis = 0)

ax_dict[fig_ax].plot(np.linspace(1, 16, 16), emg_diff_50, '-v', color = utils.colors['emg'], ms = 3)

upper_end = emg_diff_25 
lower_end = emg_diff_75 
ax_dict[fig_ax].fill_between(np.linspace(1, 16, 16), lower_end, upper_end, 
                             color = utils.colors['emg'], alpha = 0.2, edgecolor = 'None')

ax_dict[fig_ax].set_ylim(0, 70)


fig_emg

In [ ]:
td_error.shape

Extra plot just in case -- this is average error across trials, showing that error is going down across all trials

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (1.5, 1))

 
# fig_ax = 'd'
# ax = ax_dict[fig_ax]
# arrange final mean error
td_error_trials = np.zeros((utils.n_keys, utils.n_blocks, utils.n_conds))
for iK in range(utils.n_keys):
    for iB in range(utils.n_blocks):
        for iC in range(utils.n_conds):
            td_error_trials[iK, iB, iC] = np.mean(td_error[iB, iK, conds_trial_list[iB, iK, iC], :], axis = -1)

# flatten across blocks
td_error_trials_append = np.append(td_error_trials[:, 0], td_error_trials[:, 1], axis = 1)
# confirm that the data is flattened across blocks and conditions correctly
assert(td_error_trials_append.shape == (utils.n_keys, utils.n_blocks * utils.n_conds))


# # take the interquartile distribution across subjects (N = 14) 
td_25, td_50, td_75 = np.percentile(td_error_trials_append, [25, 50, 75],  axis = 0)

ax.plot(np.linspace(1, 16, 16), td_50, '-o', color = utils.colors['emg'], ms = 3)

upper_end = td_25 
lower_end = td_75 
ax.fill_between(np.linspace(1, 16, 16), lower_end, upper_end, 
                             color = utils.colors['emg'], alpha = 0.2, edgecolor = 'None')

ax.set_ylabel("Mean Error (au)")
ax.set_xlabel("Trials")
ax.set_title("Average Error Across Trials")

# fig_emg

In [ ]:
# time-domain error across time - block 1 vs block 2
fig_ax = 'd'
axs = ax_dict[fig_ax]

#find the mean error per subject per trial
# each subject has a mean block 1 and block 2 error
td_error_med = np.mean(td_error, axis = 2)
assert(td_error_med.shape == (utils.n_blocks, utils.n_keys, utils.min_time))

# find the distribution of those error across subjects
td_all25, td_all50, td_all75 = np.percentile(td_error_med, [25, 50, 75], axis=1)

assert(td_all50.shape == (utils.n_blocks, utils.min_time))

x = np.linspace(0, 5, utils.min_time)
pt = (70*5) # smoothing window = 5 seconds
x = x[utils.RAMP:-pt+1]

# BLOCK 1
axs.plot(x, plotting.smooth(td_all50[0, utils.RAMP:], pt), '-', color = utils.colors['B1'] , label='BLOCK1', lw = 1, zorder = 10)
axs.fill_between(x, plotting.smooth(td_all25[0, utils.RAMP:], pt), 
                    plotting.smooth(td_all75[0, utils.RAMP:], pt), alpha=0.2, color = utils.colors['B1'], edgecolor = 'None')

# BLOCK 2
axs.plot(x, plotting.smooth(td_all50[1, utils.RAMP:], pt), '-', color = utils.colors['B2'], label='BLOCK2', lw = 1, zorder = 10)
axs.fill_between(x, plotting.smooth(td_all25[1, utils.RAMP:], pt),
                    plotting.smooth(td_all75[1, utils.RAMP:], pt), alpha=0.2, color = utils.colors['B2'], edgecolor = 'None')

# mouse_error = 4.2
# axs.hlines(mouse_error, 0, np.max(x))
# axs.set_ylim(10, 25)

fig_emg

In [ ]:
image_format = 'pdf' # e.g .png, .svg, etc.
image_name = 'fig1-emg-learning-mean-data-cm.pdf'
PATH = '/results/'
fig_emg.savefig(PATH + image_name, format=image_format, dpi=300)